# ICE-NMF Parameter Tunning ml1m

In [1]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ml1m_gte5.csv'


Data = []
data = pd.read_csv("ml1m_gte5.csv","\t")
print(data)
Data = data.values.tolist()



        UserID  ItemID   Timestamp
0         6040     593   956703954
1         6040    2019   956703977
2         6040     213   956704056
3         6040    3111   956704056
4         6040     912   956704191
...        ...     ...         ...
226305    5950    3363  1046367948
226306    5950     111  1046368241
226307    5950    3196  1046368462
226308    5948    1180  1046437879
226309    4958    1407  1046454443

[226310 rows x 3 columns]


/home/mmelo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


# Parameter Study

## Iterations

In [4]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

iterations = [5,10,15,20,25,30]

for i in iterations:
    print("iter")
    print(i)
    num_clusters = 8
    cl_num_iterations = i
    cl_learn_rate = 0.1
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 8
    learn_rate = 0.1
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update']))    


iter
5
start time 2023-06-08 15:58:52.497354
npmean(resuls[Recall@20]) 0.06786444512115544
end time 2023-06-08 16:24:12.945155
run time 0:25:20.447801

get tuple 2.964566628450764e-06
recommend 0.04495656337340587
eval_point 1.7185522753142615e-05
update 0.002143468939406268
iter
10
start time 2023-06-08 16:24:13.117729
npmean(resuls[Recall@20]) 0.06743703965068584
end time 2023-06-08 16:51:19.137623
run time 0:27:06.019894

get tuple 2.990018241072896e-06
recommend 0.04863010044633772
eval_point 1.8134376018778058e-05
update 0.002304040182021866
iter
15
start time 2023-06-08 16:51:19.343475
npmean(resuls[Recall@20]) 0.06750351123595505
end time 2023-06-08 17:17:25.817449
run time 0:26:06.473974

get tuple 2.8540108239812437e-06
recommend 0.04483575024380443
eval_point 1.6858278030759833e-05
update 0.002399366604950503
iter
20
start time 2023-06-08 17:17:26.043068
npmean(resuls[Recall@20]) 0.06465159004418004
end time 2023-06-08 17:40:56.472961
run time 0:23:30.429893

get tuple 2.5697

## Eta

In [5]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])


etas = [0.0001, 0.001, 0.01, 0.1]

for e in etas:
    print("eta")
    print(e)
    num_clusters = 8
    cl_num_iterations = 20
    cl_learn_rate = e
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 8
    learn_rate = 0.1
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

eta
0.0001
start time 2023-06-08 18:34:54.613942
npmean(resuls[Recall@20]) 0.06414656978558374
end time 2023-06-08 18:57:46.561858
run time 0:22:51.947916

get tuple 2.561365213276594e-06
recommend 0.03624156144939296
eval_point 1.6032201375601227e-05
update 0.0024621123647274737
eta
0.001
start time 2023-06-08 18:57:46.731606
npmean(resuls[Recall@20]) 0.06779358636992106
end time 2023-06-08 19:19:59.540694
run time 0:22:12.809088

get tuple 2.5860498747008734e-06
recommend 0.03463004933889733
eval_point 1.638338975330875e-05
update 0.002450642853976613
eta
0.01
start time 2023-06-08 19:19:59.708602
npmean(resuls[Recall@20]) 0.06666666666666667
end time 2023-06-08 19:42:47.775453
run time 0:22:48.066851

get tuple 2.572727258151369e-06
recommend 0.03615776065624121
eval_point 1.6627517825991923e-05
update 0.0024520121989861847
eta
0.1
start time 2023-06-08 19:42:48.021061
npmean(resuls[Recall@20]) 0.06922974708774417
end time 2023-06-08 20:05:11.156561
run time 0:22:23.135500

get tupl

## Lambda

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

lambdas = [0,0.0001, 0.001, 0.01, 0.1]   

for l in lambdas:
    print("lambda")
    print(l)
    num_clusters = 8
    cl_num_iterations = 20
    cl_learn_rate =0.01
    cl_regularization = l
    num_factors = 160
    num_iter = 8
    learn_rate = 0.1
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lambda
0
start time 2023-06-08 20:05:11.901908
npmean(resuls[Recall@20]) 0.06970698391716237
end time 2023-06-08 20:26:55.029050
run time 0:21:43.127142

get tuple 2.6042839287289315e-06
recommend 0.032796966969165856
eval_point 1.6117495149997764e-05
update 0.002461470304368362
lambda
0.0001
start time 2023-06-08 20:26:55.189968
npmean(resuls[Recall@20]) 0.06939509584594464
end time 2023-06-08 20:48:34.830934
run time 0:21:39.640966

get tuple 2.578008475671744e-06
recommend 0.03290668283527701
eval_point 1.631252869297581e-05
update 0.0024202322687042264
lambda
0.001
start time 2023-06-08 20:48:34.991286
npmean(resuls[Recall@20]) 0.06576461802047182
end time 2023-06-08 21:10:28.458616
run time 0:21:53.467330

get tuple 2.554021234155402e-06
recommend 0.0334190391715987
eval_point 1.6277307618383373e-05
update 0.002434753093742238
lambda
0.01
start time 2023-06-08 21:10:28.618652
npmean(resuls[Recall@20]) 0.061834543591571256
end time 2023-06-08 21:32:20.499859
run time 0:21:51.881207